In [ ]:
# setup
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy import linalg

In [ ]:
# setup: has to be in a separate cell for some reason
plt.rcParams['figure.figsize'] = [10, 5]

In [ ]:
# from before
def phase_plot(genfn, xlim, ylim, nx, ny, ax, scale=False, **kwargs):
    X, Y = np.meshgrid(np.linspace(xlim[0], xlim[1], nx),
                       np.linspace(ylim[0], ylim[1], ny))
    X.shape = Y.shape = (np.prod(X.shape),)
    U, V = genfn(X, Y, **kwargs)
    if scale:
        q = ax.quiver(X, Y, U-X, V-Y, angles='xy', scale_units='xy', scale=1)
    else:
        q = ax.quiver(X, Y, U-X, V-Y, angles='xy')
    return q

def run_sim_2d(N0, M0, gen_fn, ngens, dtype='int', **kwargs):
    assert(len(N0) == len(M0))
    N = np.empty((ngens, len(N0)), dtype=dtype)
    N[0, :] = N0
    M = np.empty((ngens, len(M0)), dtype=dtype)
    M[0, :] = M0
    for t in range(1, ngens):
        N[t, :], M[t, :] = gen_fn(N[t-1, :], M[t-1, :], **kwargs)
    return N, M


# Linear algebra, age structure, and eigenpopulations

Next we'll be learning some *linear algebra*, which gives us
the tools we need to generalize what we've done so far to higher dimensions.
We'll follow Otto & Day in doing this in the context of 
*age* and *stage structured populations*.

We're also going to focus on the *short term* dynamics of these populations:
for reasons that will be clear later, we won't be concerned about long-term stability.


# Linear algebra

A quick intro on the board to:

1. Vector addition (`+`): vectors as *displacements*
2. Matrix-vector multiplication (`@`): square matrices as *transformations*
3. Matrix-matrix multiplication (`@`): *composing* two transformations
4. Matrices as systems of linear equations (`scipy.linalg.solve( )`)

Problems: let
$$
  M = \begin{bmatrix} 1 & 2 & 3 \\ 0 & 4 & 5 \\ 0 & 0 & 6 \end{bmatrix}
$$
and
$$\begin{aligned}
 x = \begin{bmatrix} 3 \\ -6 \\ 9 \end{bmatrix}
 & \qquad &
 y = \begin{bmatrix} 1 \\ 1 \\ 1 \end{bmatrix} .
\end{aligned}$$


1. Find $x + y$.
2. Find $M x$.
3. Show that $M(x+y) = Mx + My$.
4. Show that $M(Mx) = (M^2)x$.
5. Find the vector $u$ so that $Mu = x$.


# Life stages (aka class-structured populations)

Let's go back to thinking about a single species,
but keeping track not just of how many individuals there are,
but also what *life stage* or *class* they are all in.
We *could* keep track of this just as a list of life stages -
if there are $N$ individuals, this would be a list of $N$ numbers.
But it's tidier to keep track of the *census*:
for each possible class, how many individual are there of that age?
We'll write that as follows:
$$
  N_k(t) = \text{(number of class-$k$ individuals alive at time $t$)} .
$$


## Example: juveniles/adults

Let's start with the simplest example.
- Every individual is either *juvenile* (class $J$) or *adult* (class $A$).
- Juveniles *mature* into adults with probability $p_j$, or else die.
- Adults give birth to on average $b$ offspring per time step, and
- survive to the next time step with probability $p_a$.

The discrete-time (deterministic) equation is
$$ \begin{aligned}
  N_J(t+1) &= 0 N_J(t) +  b N_A(t) \\
  N_A(t+1) &= p_j N_J(t) + p_a N_A(t) .
\end{aligned} $$
Using the matrix
$$\begin{aligned}
M =
\begin{array}{c c} & \begin{array}{@{} c c @{}} J\hphantom{0} & \hphantom{0}A \end{array} \\
    \begin{array}{c} J \\ A \end{array}\hspace{-1em} &
    \left(
        \begin{array}{@{} c  @{}}
        0 & b \\
        p_j & p_a
        \end{array}
    \right) \\ \mbox{} \end{array} \\[-12pt] ,
\end{aligned}$$
and $N = (N_J, N_A)$,
we can write this same system of equations concisely as
$$
N(t+1) = M N(t) .
$$

**What's this system do?**

In [ ]:
# The parameters
b = 10
pj = 0.1
pa = 0.2
M = np.array([[0, b], [pj, pa]])
print(M)

In [ ]:
# suppose we start with one juvenile and 9 adults?
print(M @ np.array([1, 9]))

In [ ]:
# iterate these equations
N0 = [1, 9]
ntimes = 20
N = np.empty((ntimes, 2), dtype='float')
N[0,:] = N0
for j in range(1, ntimes):
    N[j, :] = M @ N[j-1, :]
    
fig, ax = plt.subplots()
ax.plot(N)
ax.set_xlabel("time")
ax.set_ylabel("numbers")
ax.legend(['juveniles', 'adults']);

# Eigendirections

*Eigenvectors* of a matrix are directions *unchanged* by its transformation.
In symbols, this is
$$
  Mv = \lambda v
$$
where $\lambda$ is a *number*, called the *eigenvalue*.

Exercises:

1. Check that $(1, 0)$ and $(0, 1)$ are eigenvectors of
    $$
    M = \begin{bmatrix} 2 & 0 \\ 0 & 3 \end{bmatrix} .
    $$
    What are the eigenvalues?

2. Use `scipy.linalg.eig( )` to find the eigenvalues and vectors of 
    $$
      M = \begin{bmatrix} 1 & 2 & 3 \\ 0 & 4 & 5 \\ 0 & 0 & 6 \end{bmatrix}
    $$
    Double-check the equation holds.

3. The formula corresponding to "change into the coordinates of the eigenvectors,
    multiply by the eigenvalues, and change coordinates back" is
    $$
      M = U \text{diag}(\lambda) U^{-1} ,
    $$
    where $U^{-1}$ is the *inverse matrix* (`scipy.linalg.solve(U, np.eye(U.shape[0]))`)
    and $\text{diag}(\lambda)$ is the matrix with the values of $\lambda$ on the diagonal
    and zeros elsewhere.
    Check this holds.


In [ ]:
lam, U = linalg.eig(M)
U_inv = linalg.solve(U, np.eye(U.shape[0]))
print(U @ np.diag(lam) @ U_inv)
print(M)
print("U:", U)
print("diag(lam)", np.diag(lam))
print("U inverse:", U_inv)

### Example: eigenvectors

Here's the vector field defined by the transformation we get from
$$
  M = \begin{bmatrix} 2 & 1 \\ 1/2 & 1 \end{bmatrix} ,
$$
with the eigenvector directions superimposed.

In [ ]:
M = np.array([[2, 1], [1/2, 1]])
lam, U = linalg.eig(M)

def lin_transform(X, Y, M):
    out = M @ np.row_stack([X, Y])
    return out[0, :], out[1, :]

plt.rcParams['figure.figsize'] = [8, 8]
fig, ax = plt.subplots()
phase_plot(lin_transform, [-1, 1], [-1, 1], 20, 20, ax=ax, M=M)
plt.axis("equal")
for x, y in U.T:
    ax.plot([-x, x], [-y, y])

fig.legend(['evector1', 'evector2']);

## Eigenanalysis of a stage-structured model

We're going to find the time course of the linear model
$$
   N(t) = M^t N(0) ,
$$
using the eigendecomposition of $M$.
If we can do an eigendecomposition, it looks like 
$$
  M = U \text{diag}(\lambda) U^{-1} ,
$$
and so
$$
  M^t = U \text{diag}(\lambda^t) U^{-1} .
$$

Written out, this says that to find the vector $N(t)$, we need to

1. Find the eigenvector loadings of the initial state:
   $$ V(0) = U^{-1} N(0) . $$
   These say how strongly the initial state "excites" each of the eigendirections.
  
2. Multiply each of these loadings by the corresponding eigenvalue, $\lambda$:
   $$ V(t)_k = \lambda_k^t V(0)_k . $$
   These give the contributions of each eigendirection at time $t$.
  
3. Shift back to the original coordinates by multiplying by the matrix of eigenvectors, $U$:
   $$ N(t) = U V(t) .$$



### Eigenanalysis of the juvenile-adult example

We'll apply this to the juvenile-adult model above.

In [ ]:
# The parameters
b = 10
pj = 0.1
pa = 0.2
M = np.array([[0, b], [pj, pa]])
lam, U = linalg.eig(M)  # eigenvalue, vector

# initial conditions
N0 = [1, 9]

ntimes = 20
V = np.empty((ntimes, 2), dtype='complex')
V[0, :] = linalg.solve(U, N0)
for j in range(1, ntimes):
    V[j, :] = np.power(lam, j) * V[0, :]
   
N = V @ U.T
    
fig, ax = plt.subplots()
ax.plot(np.real(N))
ax.set_xlabel("time")
ax.set_ylabel("numbers")
ax.legend(['juveniles', 'adults']);


In [ ]:
fig, ax = plt.subplots()
ax.plot(np.real(V))
ax.set_xlabel("time")
ax.set_ylabel("contribution")
ax.legend(['evector0', 'evector1']);
print(U)


## Example: right whales

For instance, the classes for females in a right whale population model were:

- class `C`: calves
- class `I`: immature
- class `M`: mature
- class `R`: reproductive

Individuals transition from calf to immature in one year;
can stay immature for more than one year;
then transition between mature and reproductive as adults.
Each class has a separate death rate,
and only reproductive individuals produce new offspring.
The parameters of the model are concisely encoded
in the following matrix:
$$
S =
\begin{array}{c c} & \begin{array}{@{} c c c c @{}} C\hphantom{0} & I\hphantom{0} & \hphantom{0}M & \hphantom{0}R \end{array} \\
    \begin{array}{c} C \\ I \\ M \\ R \end{array}\hspace{-1em} &
    \left(
        \begin{array}{@{} c c c c @{}}
        0 & 0 & 0 & b \\
        s_{IC} & s_{II} & 0 & 0 \\
        0 & s_{MI} & s_{MM} & s_{MR} \\
        0 & s_{RI} & s_{RM} & s_{RR} 
        \end{array}
    \right) \\ \mbox{} \end{array} \\[-12pt] 
$$
There are nine parameters.
All the $s_{XY}$ parameters have the same interpretation: they give the probability
that a given individual *currently in class $Y$* will be *in class $X$* next time step.
For instance, a calf will either
- become immature (with probability $s_{IC}$, or
- die (otherwise),

while a mature individual will either
- stay immature (probability $s_{II}$),
- become mature (probability $s_{MI}$), or
- become reproductive ($s_{RI}$).

The remaining parameter, $b$, is the mean number of *births* per reproductive individual per year.

In [ ]:
whale_args = {
    "s_ic" : 0.92,
    "s_ii" : 0.86,
    "s_mi" : 0.08,
    "s_mm" : 0.8,
    "s_mr" : 1 - 0.12,
    "s_ri" : 0.02,
    "s_rm" : 0.19,
    "s_rr" : 0,
    "b" : 0.3
}

In [ ]:
def make_S(s_ic, s_ii, s_mi, s_mm, s_mr, s_ri, s_rm, s_rr, b):
    S = np.array([[0, 0, 0, b],
                  [s_ic, s_ii, 0, 0],
                  [0, s_mi, s_mm, s_mr],
                  [0, s_ri, s_rm, s_rr]])
    return S

S = make_S(**whale_args)

evals, evecs = linalg.eig(S)
print(evals)

def top_eig(b):
    # compute and return the largest eigenvalue from S
    args = whale_args
    args['b'] = b
    S = make_S(**args)
    evals, evecs = linalg.eig(S)
    return evals[np.argmax(evals)]

bvals = np.linspace(0.01, 1.0, 20)
eigs = [top_eig(b) for b in bvals]

plt.scatter(bvals, np.real(eigs))
plt.xlabel("fecundity (b)")
plt.ylabel("top eigenvalue")
plt.axhline(1.0);

That predicts that at $b=0.2$, the population will die out (slowly, decreasing to $\approx$ 99% per year),
and above $b=0.3$ it will not (again, increasing slowly).
Let's see.

In [ ]:
# initial conditions
N0 = [10, 10, 10, 20]

ntimes = 40
N = np.empty((ntimes, S.shape[0]), dtype='float')
N[0, :] = N0
for j in range(1, ntimes):
    N[j, :] = S @ N[j-1, :]
   
  
fig, ax = plt.subplots()
ax.plot(N)
ax.set_xlabel("time")
ax.set_ylabel("numbers")
ax.legend(['juvenile', 'immature', "mature", "reproductive"]);

# over this time we expect the population to have grown by a factor of
np.real(evals[0]) ** ntimes

In [ ]:
bad_whale_args = whale_args
bad_whale_args['b'] = 0.2
S_bad = make_S(**bad_whale_args)
bevals, bevecs = linalg.eig(S_bad)

N0 = [100, 100, 100, 100]
ntimes = 200
bN = np.empty((ntimes, S.shape[0]), dtype='float')
bN[0, :] = N0
for j in range(1, ntimes):
    bN[j, :] = S_bad @ bN[j-1, :]
   
  
fig, ax = plt.subplots()
ax.plot(bN)
ax.set_xlabel("time")
ax.set_ylabel("numbers")
ax.legend(['juvenile', 'immature', "mature", "reproductive"]);

# over this time we expect the population to have grown by a factor of
np.real(bevals[np.argmax(bevals)]) ** ntimes

In [ ]:
print(evecs[:,0] / sum(evecs[:,0]))
print(linalg.eig(S.T))

## Stable age distributions

Clearly, there's some nonequilibrium stuff going on at the start of those simulations.
I chose starting age distributions that weren't natural at all -- too many juveniles,
given the long lifetimes of adults and relatively low birth rates.
How could we have predicted this?

It turns out that in a linear model, the long-term behavior is governed
by the "top" eigenvalue and its eigenvectors.
Call $\lambda_0$ the largest eigenvalue (largest in absolute value; it is always real);
and $v$ the corresponding eigenvector.
Then

1. The long-term growth rate of the entire population is $\lambda_0$,
    i.e., $\sum_j N_j(t)$ grows like $\lambda_0^t$.
   
2. The long-term *age distribution* is proportional to $v$,
    i.e., $$ \frac{N_j(t) }{ \sum_k N_k(t) } \approx \frac{ v_j }{ \sum_k v_k }.$$

Let's check this in the whale model above.

In [ ]:
evecs[:, 0]

In [ ]:
# good parameters
fig, ax = plt.subplots()
ax.hlines(evecs[:, 0]/sum(evecs[:, 0]), xmin=-5, xmax=45)
ax.plot(N / N.sum(axis=1, keepdims=True));

In [ ]:
# good parameters
fig, ax = plt.subplots()
ax.hlines(bevecs[:, 0]/sum(bevecs[:, 0]), xmin=-5, xmax=45)
ax.plot(bN / bN.sum(axis=1, keepdims=True));

# Homework

Modify the whale model to include randomness. See how well the growth rate and long-term age structure match the deterministic predictions. (They should match quite well!)

# Application to nonlinear systems

The theory above lets us analyze *nonlinear* systems by *linearizing* about fixed points.
We'll do this in a simple **predator-prey** model:

- $\theta$ : prey immigration rate
- $a$ : predator-prey contact (and consumption) rate
- $\epsilon$ : predator conversion efficiency
- $\delta$ : predator death rate

The deterministic differential equations are
$$\begin{aligned}
\frac{d n_1}{dt} &= \theta - a n_1 n_2 \\
\frac{d n_2}{dt} &= \epsilon a n_1 n_2 - \delta n_2 .
\end{aligned}$$

**Exercise:** which is the prey and which is the predator?

## Stable points

To get stable points, we need to solve for when predator and prey numbers do not change.
Predator numbers do not change if $dn_2/dt = 0$, i.e., if
$$
   n_2 (\epsilon a n_1 - \delta) = 0 .
$$
This happens if either $n_2 = 0$ (naturally),
or if $n_1 = \delta / (\epsilon a)$.
Prey numbers do not change if $dn_1/dt = 0$, i.e., if
$$
  n_2 = \theta / (a n_1) .
$$
Putting this together, we have two equilibria: zero, and
$$\begin{aligned}
  n_1^* &= \frac{\delta}{\epsilon a} \\
  n_2^* &= \frac{\theta \epsilon}{\delta} .
\end{aligned}$$


In [ ]:
def pp_eqn(N, M, dt, theta, a, eps, delta):
    next_N = N + dt * (theta - a * N * M)
    next_M = M + dt * (eps * a * N * M - delta * M)
    return next_N, next_M

def pp_equil(theta, a, eps, delta):
    return delta / (eps * a), theta * eps / delta

pp_args = {'theta' : 10,
           'a' : 0.1,
           'eps' : 1.0,
           'delta' : 0.3}
equil = pp_equil(**pp_args)

tN, tM = run_sim_2d([20], [2], pp_eqn, 200, dtype='float',
                    dt=0.1, **pp_args)


plt.rcParams['figure.figsize'] = [10, 10]
fig, ax = plt.subplots()
phase_plot(pp_eqn, xlim=[0, 40], ylim=[0, 40], nx=20, ny=20, scale=True,
           ax=ax, dt=0.1, **pp_args)
ax.plot(tN, tM)
ax.scatter(tN, tM)
ax.scatter(equil[0], equil[1], s=1000, c='g', alpha=0.5)
ax.axis('equal')
plt.xlabel("prey")
plt.ylabel("predators");


### Local approximations

Ok, so what's going on near that equilibrium?
Let's reparameterize, so that $x_1$ is the deviation of numbers of prey from equilibrium,
and $x_2$ is the deviation of the number of predators:
$$\begin{aligned}
  n_1 &= n_1^* + x_1 \\
  n_2 &= n_2^* + x_2
\end{aligned}$$
The deterministic differential equations are, rewritten:
$$\begin{aligned}
\frac{d x_1}{dt} &= \theta - a (n_1^* + x_1) (n_2^* + x_2) \\
                 &= \theta - a n_1^* n_2^* - a (n_1^* x_2 + x_1 n_2^* + x_1 x_2) \\
                 &= - a (n_1^* x_2 + x_1 n_2^* + x_1 x_2) \\
                 &\approx - a (n_1^* x_2 + x_1 n_2^*) \\
                 &= - \frac{a \epsilon \theta }{ \delta } x_1 - \frac{ \delta }{ \epsilon } x_2 \\
\frac{d x_2}{dt} &= \left(\epsilon a (n_1^* + x_1) - \delta \right) (n_2^* + x_2) \\
                 &= \left(\epsilon a n_1^* - \delta\right) n_2^* + \epsilon a x_1 n_2^* - \delta x_2 + a x_1 x_2 \\
                 &\approx \left(\epsilon a n_1^* - \delta \right) x_2 + \epsilon a x_1 n_2^* \\
                 &= \frac{\epsilon^2 a \theta }{ \delta} x_1 .
\end{aligned}$$
The steps here are (1) use the fact that $(n_1^*, n_2^*)$ is an equilibrium (so some terms become zero),
and (2) only keeping terms to *first order* in $x_1$ and $x_2$.

This says that, if the system is near the nonzero equilibrium,
it acts like the *linear* system whose matrix is
$$
  J = \begin{bmatrix} 
    - \frac{a \epsilon \theta }{ \delta } & - \frac{ \delta }{ \epsilon } \\
     \frac{\epsilon^2 a \theta }{ \delta} & 0 
   \end{bmatrix}
$$

In [ ]:
def pp_approx_eqn(N, M, dt, theta, a, eps, delta):
    equil = pp_equil(theta, a, eps, delta)
    J = np.array([[- a * eps * theta / delta, - delta / eps], [eps * eps * a * theta / delta, 0]])
    old_X = N - equil[0]
    old_Y = M - equil[1]
    dX, dY = dt * J @ np.row_stack([old_X, old_Y])
    next_N = N + dX
    next_M = M + dY
    return next_N, next_M

pp_args = {'theta' : 10,
           'a' : 0.1,
           'eps' : 1.0,
           'delta' : 0.3}
equil = pp_equil(**pp_args)

W = 3.0
tN, tM = run_sim_2d([equil[0] + W], [equil[1]], pp_eqn, 200, dtype='float',
                    dt=0.1, **pp_args)
aN, aM = run_sim_2d([equil[0] + W], [equil[1]], pp_approx_eqn, 200, dtype='float',
                    dt=0.1, **pp_args)

plt.rcParams['figure.figsize'] = [10, 10]
fig, ax = plt.subplots()
phase_plot(pp_eqn, xlim=[equil[0] - W, equil[0] + W], 
           ax=ax, ylim=[equil[1] - W, equil[1] + W], nx=20, ny=20, scale=True,
           dt=0.1, **pp_args)
q = phase_plot(pp_approx_eqn, xlim=[equil[0] - W, equil[0] + W], 
               ax=ax, ylim=[equil[1] - W, equil[1] + W], nx=20, ny=20, scale=False,
               dt=0.1, **pp_args)
q.set(color = 'm')
ax = fig.gca()
ax.plot(tN, tM)
ax.scatter(tN, tM)
ax.plot(aN, aM)
ax.scatter(aN, aM)
ax.scatter(equil[0], equil[1], s=1000, c='g', alpha=0.5)
ax.axis('equal')
plt.xlabel("prey")
plt.ylabel("predators");


## Stability analysis

Let's see now how the eigenvalues of $J$ change as we adjust $\delta$.

- If all eigenvalues have negative Real part, the equilibrium is a *sink*.
- If any eigenvalues have nonzero Imaginary part, the equilibrium has an oscillatory component.

In [ ]:
def pp_eigs(theta, a, eps, delta):
    equil = pp_equil(theta, a, eps, delta)
    J = np.array([[-a * eps * theta / delta, - delta / eps], [eps * eps * a * theta / delta, 0]])
    evals, evecs = linalg.eig(J)
    return evals

dvals = np.linspace(0.3, 2.0, 400)
J_eigs = np.array([pp_eigs(pp_args['theta'], pp_args['a'], pp_args['eps'], delta)
                   for delta in dvals])

fig, ax = plt.subplots()
ax.plot(np.real(J_eigs[:, 0]), np.imag(J_eigs[:, 0]))
sc = ax.scatter(np.real(J_eigs[:, 0]), np.imag(J_eigs[:, 0]), 
                c=dvals, cmap=matplotlib.cm.jet, label='delta')
ax.plot(np.real(J_eigs[:, 1]), np.imag(J_eigs[:, 1]))

sc = ax.scatter(np.real(J_eigs[:, 1]), np.imag(J_eigs[:, 1]), 
                c=dvals, cmap=matplotlib.cm.jet, label='delta')
fig.colorbar(sc)
plt.xlabel("Real(evalue)")
plt.ylabel("Imag(evalue)");

**Ah-ha:** there is some value of $\delta$ at which point the two eigenvalues hit each other,
then move off in the imaginary axis!  For $\delta$ above that point,
the system should oscillate as it converges.

**Exercise:** Find the value of $\delta$ at which the top eigenvalue of $J$ leaves the real line.

In [ ]:
# This should oscillate

pp_args = {'theta' : 10,
           'a' : 0.1,
           'eps' : 1.0,
           'delta' : 1.95}
equil = pp_equil(**pp_args)

W = 2.0
tN, tM = run_sim_2d([equil[0] + W], [equil[1]], pp_eqn, 200, dtype='float',
                    dt=0.1, **pp_args)
aN, aM = run_sim_2d([equil[0] + W], [equil[1]], pp_approx_eqn, 200, dtype='float',
                    dt=0.1, **pp_args)

plt.rcParams['figure.figsize'] = [10, 10]
fig, ax = plt.subplots()
phase_plot(pp_eqn, xlim=[equil[0] - W, equil[0] + W], 
           ax=ax, ylim=[equil[1] - W, equil[1] + W], nx=20, ny=20, scale=True,
           dt=0.1, **pp_args)
q = phase_plot(pp_approx_eqn, xlim=[equil[0] - W, equil[0] + W], 
               ax=ax, ylim=[equil[1] - W, equil[1] + W], nx=20, ny=20, scale=False,
               dt=0.1, **pp_args)
q.set(color = 'm')
ax = fig.gca()
ax.plot(tN, tM)
ax.scatter(tN, tM)
ax.plot(aN, aM)
ax.scatter(aN, aM)
ax.scatter(equil[0], equil[1], s=1000, c='g', alpha=0.5)
ax.axis('equal')
plt.xlabel("prey")
plt.ylabel("predators");
